In [1]:
import os
import csv
import pymysql as mc 
import time
import webbrowser
import sys
from sklearn.cluster import KMeans
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, learning_curve
from sklearn.preprocessing import scale, PolynomialFeatures

In [2]:
cnx = mc.connect(user='akashgoyal',password='*****',
                 host='stromberg.cs.uchicago.edu',database='mlb_practicum')

In [4]:
data_2017 = Akash_Customized(cnx,year=2017)

done running query
done adding columns
done preparing new df for regression


In [5]:
X = data_2017.analysis_df.drop(['1'],axis=1)
Y = data_2017.Y
clf = LogisticRegression(penalty='l1')
clf.fit(X,Y)
clf.score(X,Y)

0.7665415625090585

In [8]:
heads = list(X.columns.values)
for i,item in enumerate(clf.coef_[0]):
    print (heads[i],item.round(2))

current_lead 0.15
mod inning 0.07
outs -0.02
current_lead^2 -0.01
current_lead mod inning 0.1
current_lead outs 0.03
mod inning^2 -0.0
mod inning outs -0.02
outs^2 -0.03
count_type 0.05
100rc 5.68
300rc 0.59
500rc 2.51
1000rc -1.25
2000rc 3.54
allRc -3.83
1b runner 0.14
2b runner 0.24
3b runner 0.29
homeAway -0.54


In [74]:
heads = list(X.columns.values)
for i,item in enumerate(clf.coef_[0]):
    print (heads[i],item.round(2))

current_lead 0.17
mod inning 0.04
outs -0.04
current_lead^2 0.0
current_lead mod inning 0.1
current_lead outs 0.03
mod inning^2 -0.0
mod inning outs -0.02
outs^2 -0.02
count_type 0.06
100rc 1.99
300rc -1.43
500rc 0.48
1000rc 0.25
2000rc 0.08
allRc 0.31
1b runner 0.15
2b runner 0.27
3b runner 0.32
homeAway -0.54


In [6]:
data_2015 = Akash_Customized(cnx,year=2015)

done running query
done adding columns
done preparing new df for regression


In [7]:
X_2015 = data_2015.analysis_df.drop(['1'],axis=1)
Y_2015 = data_2015.Y
clf.score(X_2015,Y_2015)

0.7680908004574776

In [59]:
easy_pickings = data_2015.analysis_df
easy_pickings['Outcome'] = data_2015.Y.values
easy_pickings = easy_pickings[(abs(easy_pickings['mod inning'])>12) & (
                                abs(easy_pickings['current_lead'])<1)]
X = easy_pickings.drop(['Outcome'],axis=1)
Y = easy_pickings['Outcome']
logit.score(X,Y)

0.6026038437693738

In [3]:
class WinProb:
    def __init__(self,cnx,year=None):
        self.cnx = cnx
        self.year = year
        self.df = self.run_sql_query()
        print ('done running query')
        self.add_relevant_columns()
        print ('done adding columns')
        self.analysis_df,self.Y = self.prepare_logistic_regression()
        print ('done preparing new df for regression')
    
    def prepare_logistic_regression(self):
        X = self.df[['current_lead','mod inning','outs']]
        Y = self.df['win']
        interactions = PolynomialFeatures(degree=2)
        newX = interactions.fit_transform(X)
        nameList = interactions.get_feature_names(X.columns)
        nameList.extend(['count_type','100rc','300rc','500rc','1000rc',
                         '2000rc','allRc','1b runner','2b runner','3b runner','homeAway'])
        count_type = self.df['count_type'].values.reshape(len(self.df),1)
        war100 = self.df['100_avg'].values.reshape(len(self.df),1)
        war300 = self.df['300_avg'].values.reshape(len(self.df),1)
        war500 = self.df['500_avg'].values.reshape(len(self.df),1)
        war1000 = self.df['1000_avg'].values.reshape(len(self.df),1)
        war2000 = self.df['2000_avg'].values.reshape(len(self.df),1)
        warAll = self.df['all_avg'].values.reshape(len(self.df),1)
        oneb = self.df['1b runner'].values.reshape(len(self.df),1)
        twob = self.df['2b runner'].values.reshape(len(self.df),1)
        threeb = self.df['3b runner'].values.reshape(len(self.df),1)
        homeAway = self.df['home_away'].values.reshape(len(self.df),1)
        
        X = np.hstack((newX,count_type,war100,war300,war500,war1000,war2000,warAll,oneb,twob,threeb,homeAway))
        analysis_df = pd.DataFrame(X,columns=nameList)
        return analysis_df,Y
        
    def run_sql_query(self):
        query = ''
        if not self.year:
            query = """SELECT Game.gameID AS gameID,abNum,pitchID,the_event,
                        firstBaseRunner,secondBaseRunner, thirdBaseRunner,outs,
                        home_team_runs,away_team_runs,curr_inn,balls,strikes, homeTeamScore,
                        awayTeamScore,gameDate,100_avg,300_avg,500_avg,1000_avg,2000_avg, all_avg
                        FROM Pitch2 INNER JOIN Batter_Run_Expectancy ON 
                        (Pitcher_Run_Expectancy.gameID = Pitch2.gameID AND Pitcher_Run_Expectancy.playerID = Pitch2.pitcherID)"""
        else:
            query = """SELECT Game.gameID AS gameID,abNum,pitchID,the_event,
                        firstBaseRunner,secondBaseRunner, thirdBaseRunner,outs,
                        home_team_runs,away_team_runs,curr_inn,balls,strikes, homeTeamScore,
                        awayTeamScore,gameDate,100_avg,300_avg,500_avg,1000_avg,2000_avg, all_avg
                        FROM Pitch2 INNER JOIN Game ON Pitch2.gameID = Game.gameID
                        INNER JOIN Pitcher_Run_Expectancy ON 
                        (Pitcher_Run_Expectancy.gameID = Pitch2.gameID AND Pitcher_Run_Expectancy.playerID = Pitch2.pitcherID)
                        WHERE YEAR(gameDate)={}""".format(self.year)
            df = pd.read_sql_query(query,cnx)
            return df 
    
    def add_relevant_columns(self):
        self.df['win'] = pd.Series([(1 if (x[0]<x[1] and x[2]*2%2==0) or (x[0]>x[1] and x[2]*2%2==1) else 0)
                        for x in self.df[['homeTeamScore','awayTeamScore','curr_inn']].values])

        #add the current lead 
        l = []
        for x in self.df[['home_team_runs','away_team_runs','curr_inn']].values:
            #if home team
            if x[2]*2%2==1:
                l.append(x[0]-x[1])
            else:
                l.append(x[1]-x[0])
        self.df['current_lead'] = l

        #add count classification (bad,neutral,favorable)
        l = []
        errs= []
        good = [(3,0),(3,1),(2,0),(3,2)]
        neutral = [(0,0),(1,0),(2,1),(1,1),(2,2)]
        bad = [(1,2),(0,2),(0,1)] 
        for x in self.df[['balls','strikes']].values:
            count = (x[0],x[1])
            if count in good:
                l.append(2)
            elif count in neutral:
                l.append(1)
            elif count in bad:
                l.append(0)
            else:
                errs.append(count)
        self.df['count_type'] = l
        
        #indicator variables if runners on base
        firstRunner = []
        secondRunner = []
        thirdRunner = []
        mod_inning = []
        home_away = []
        for x in self.df[['firstBaseRunner','secondBaseRunner','thirdBaseRunner','curr_inn']].values:
            firstRunner.append(1 if x[0] else 0)
            secondRunner.append(1 if x[1] else 0)
            thirdRunner.append(1 if x[2] else 0)
            if (2*x[3])%2 == 1:
                mod_inning.append(x[3]-0.5)
                home_away.append(0)
            else:
                mod_inning.append(x[3])
                home_away.append(1)
        self.df['1b runner'] = firstRunner
        self.df['2b runner'] = secondRunner
        self.df['3b runner'] = thirdRunner
        self.df['mod inning'] = mod_inning
        self.df['home_away'] = home_away
        
        

In [35]:
def print_url(gameID,cnx):
    cursor = cnx.cursor()
    cursor.execute("SELECT gameID, homeTeam,awayTeam,YEAR(gameDate), MONTH(gameDate),DAY(gameDate),isDoubleHeader FROM Game WHERE gameID='{}'".format(gameID))
    
    theURL=''
    for row in cursor:
        theYear = str(row[3])
        theMonth= 0
        theDate=0
        if row[4]<10:
            theMonth ='0'+str(row[4])
        else:
            theMonth=str(row[4])
        if row[5]<10:
            theDate='0'+str(row[5])
        else:
            theDate=str(row[5])

        gameNum= '1'
        if str(row[6])==2:
            gameNum='2'
        homeTeam=row[1].lower()+'mlb'
        visitTeam=row[2].lower()+'mlb'
        new_path = 'year_'+theYear+'/month_'+theMonth + '/day_'+theDate+'/gid_'+theYear+'_'+theMonth +'_'+theDate+'_'
        new_path = new_path+visitTeam+'_'+homeTeam+'_'+gameNum+'/inning/inning_all.xml'
        theURL='http://gd2.mlb.com/components/game/mlb/'+new_path
    print (theURL)